The following is an attempt to use Google's Tensorflow ML library to run a NN classifier. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import random

voice =pd.read_csv('../input/voice.csv')
voice = pd.DataFrame(voice)
print(voice.head())
# Any results you write to the current directory are saved as output.

Increasing dataset size via adding small distortions may help training, but not necessarily. The problem here is that all original data is used for training. Permutation is necessary for a correct validation. It is verified that adding noise does not help in this case.

In [ ]:
#for i in range(6):
#	copy = voice
#	copy['meanfreq']=copy['meanfreq']+random.gauss(.0001,.001) # add noice to mean freq var
#	voice=voice.append(copy,ignore_index=True) # make voice df 2x as big
#	print("shape of df after {0}th intertion of this loop is {1}".format(i,voice.shape))

#voice.apply(np.random.permutation)


In [ ]:
label = voice.pop("label")

# converts from dataframe to np array
voice=voice.values

# --------------- Pre-processing: feature selection ---------------------
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# some features are redundant as shown by correlation, therefore, they are dropped.
X2 = np.delete(voice, [7, 11, 17] , 1)  

# data standarization (zero-mean, unit variance) ~ truncation to [-1, 1]
scaler = StandardScaler()
scaler.fit(X2)
X2 = scaler.transform(X2)

# dimension reduction using PCA
pca = PCA(n_components=16)
pca.fit(X2)
X2 = pca.transform(X2) 
nFeature = X2.shape[1]

# convert train labels to one hots: tf.one_hot()
train_labels = pd.get_dummies(label).values

TestPortion = 0.2

# It is important to take same test_size and random_state for comparison
x_train,x_test,y_train,y_test = train_test_split(X2, train_labels, test_size=TestPortion, random_state=1)

Now some preparation for tf session: Placeholders for data to be feed in. 
Then we build our NN with one hidden layer.

In [ ]:
# place holder for inputs. feed in later
x = tf.placeholder("float", [None, nFeature])

# take 'nFeautre' features to 'N2' nodes in hidden layer
N2 = 100
w1 = tf.Variable(tf.truncated_normal([nFeature, N2], stddev=0.05, name = 'w1')) # tf.Variable(tf.random_normal([nFeature, N2], stddev=.05, name='w1'))
b1 = tf.Variable(tf.constant(0., shape=[N2])) #b1 = tf.Variable(tf.zeros([N2]))

# calculate activations 
#hidden_output = tf.nn.softmax(tf.matmul(x, w1) + b1)
hidden_output = tf.nn.relu(tf.matmul(x, w1) + b1)

# bring from 'N2' nodes to 2 for output: init weights with truncated normal distribution
w2 = tf.Variable(tf.truncated_normal([N2, 2], stddev=0.025, name = 'w2')) 
b2 = tf.Variable(tf.constant(0., shape=[2])) 

# placeholder for labels
y_ = tf.placeholder("float", [None,2])

# #implement model. these are predicted ys
#y = tf.nn.softmax(tf.matmul(hidden_output, w2) + b2)
y = tf.nn.sigmoid(tf.matmul(hidden_output, w2) + b2)

In [ ]:
loss = tf.reduce_mean(tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y, y_, name='xentropy')))

Create an optimizer op, and a train step which minimizes the weight and bias vars. 

In [ ]:
opt = tf.train.AdamOptimizer(learning_rate=1e-4)
train_step = opt.minimize(loss, var_list=[w1,b1,w2,b2])

Create a way to assess the accuracy of our prediction:

In [ ]:
tf_correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
tf_accuracy = tf.reduce_mean(tf.cast(tf_correct_prediction, "float"))

Function to get batches for training. Start tf session and initialize variables. 

In [ ]:
def get_mini_batch(x,y):
	rows=np.random.choice(x.shape[0], 100)
	return x[rows], y[rows]

In [ ]:
# Start an interactive session
sess = tf.InteractiveSession()
summary_writer = tf.train.SummaryWriter('voices')
# summary_writer = tf.train.SummaryWriter('voices', sess.graph)

# # init all vars
init = tf.global_variables_initializer()
sess.run(init)

Now, it is time to train. Run a loop for ntrials iterations. Feed the training data into the train_step. Then print our accuracy on the test set and the corresponding value of the loss function.

In [ ]:
ntrials = 10000
for i in range(ntrials):
    # get mini batch
    a,b=get_mini_batch(x_train,y_train)
    # run train step, feeding arrays of 100 rows each time
    _, cost =sess.run([train_step,loss], feed_dict={x: a, y_: b})
    if i%100 ==0:
        trainAccuracy = tf_accuracy.eval(feed_dict={x: a, y_: b})  
        print("epoch is {0} and cost is {1} with train accuracy {2}".format(i, cost, trainAccuracy))    	

Check accuracy on test set and close the session

In [ ]:
result = tf_accuracy.eval(feed_dict={x: x_test, y_: y_test})

print("Test accuracy: {}".format(result))
sess.close()